In [1]:
import pandas as pd
import os
import datetime as dt
import numpy as np
from statistics import mode
import statistics as st

pd.set_option('display.max_columns', None)
#pd.reset_option('max_columns')

In [2]:
print(os.path.abspath('.'))

D:\jupyter\stock_price


In [3]:
# 匯入、合併多日資料，產生 raw_book_train、raw_raw_trade_train

# train_date = ['01','02','03','07','08','09','10','13','14','15','16']
# test_date = [17,20,21,22,23,24,27,28,29,30]

train_date = ['01','02','03','07','08','09']
test_date = [17,20,21]

# 五檔報價資料
com_train_book_data = pd.DataFrame()
for i in train_date:
    raw_data = pd.read_parquet(f'book/600519/202004{i}.parquet.gzip')
    com_train_book_data = pd.concat([com_train_book_data,raw_data])

com_test_book_data = pd.DataFrame()
for i in test_date:
    raw_data = pd.read_parquet(f'book/600519/202004{i}.parquet.gzip')
    com_test_book_data = pd.concat([com_test_book_data,raw_data])

com_train_book_data['model'] = 1
com_test_book_data['model'] = 0

raw_book_train = pd.concat([com_train_book_data, com_test_book_data])  #book_train


# 交易量資料
com_train_trade_data = pd.DataFrame()
for i in train_date:
    raw_data = pd.read_parquet(f'trade/600519/202004{i}.parquet.gzip')
    com_train_trade_data = pd.concat([com_train_trade_data,raw_data])

com_test_trade_data = pd.DataFrame()
for i in test_date:
    raw_data = pd.read_parquet(f'trade/600519/202004{i}.parquet.gzip')
    com_test_trade_data = pd.concat([com_test_trade_data,raw_data])

com_train_trade_data['model'] = 1  #訓練資料
com_test_trade_data['model'] = 0  #預測資料

com_test_trade_data['nextMidpt'] = np.nan  #要預測的值，先設None

raw_trade_train = pd.concat([com_train_trade_data, com_test_trade_data])  #trade_train

In [4]:
# data1 = pd.DataFrame({'A':pd.Series({1:1,2:2,3:3,4:4,5:5})}, columns=['A'])
# data2 = pd.DataFrame({'A':pd.Series({1:11,2:12,3:13})},columns=['A'])
# pd.concat([data1,data2])

In [5]:
## 五檔報價(特徵) -----------------------------------------------------------------------------------------
# raw_book_train = pd.read_parquet('book/600519/20200401.parquet.gzip')
book_train = raw_book_train.copy()

In [6]:
book_train['TimeStamp2'] = pd.to_datetime(book_train['TimeStamp'].values, unit='us')
book_train.head()

,TimeStamp,Bid1,Bid1Size,Ask1,Ask1Size,Bid2,Bid2Size,Ask2,Ask2Size,Bid3,Bid3Size,Ask3,Ask3Size,Bid4,Bid4Size,Ask4,Ask4Size,Bid5,Bid5Size,Ask5,Ask5Size,midpt,nextMidpt,model,TimeStamp2
index,,,,,,,,,,,,,,,,,,,,,,,,,
137,1585704603505567,1113.00,1900,1115.03,100,1112.22,2800,1116.00,500,1112.00,600,1117.0,100,1111.21,3000,1117.99,8358,1111.11,1600,1118.0,1300,1114.015,1115.265,1,2020-04-01 01:30:03.505567
138,1585704606961840,1115.03,100,1115.50,100,1114.44,200,1117.00,940,1113.01,1000,1117.1,3200,1113.00,1900,1117.40,4000,1112.22,2800,1117.5,100,1115.265,1116.495,1,2020-04-01 01:30:06.961840
139,1585704609513298,1116.00,1600,1116.99,100,1115.03,300,1117.00,1240,1114.44,200,1117.1,3200,1113.01,600,1117.40,4000,1113.00,2000,1117.5,100,1116.495,1113.795,1,2020-04-01 01:30:09.513298
140,1585704612124381,1113.15,300,1114.44,400,1113.01,600,1116.99,100,1113.00,2000,1117.1,140,1112.22,2800,1117.40,4000,1112.00,1600,1117.5,100,1113.795,1111.565,1,2020-04-01 01:30:12.124381
141,1585704615123840,1111.51,100,1111.62,2200,1111.50,200,1116.00,100,1111.30,100,1116.5,100,1111.21,3000,1117.00,300,1111.18,200,1117.4,4000,1111.565,1114.200,1,2020-04-01 01:30:15.123840


In [7]:
## 時間
book_train['Week'] = book_train['TimeStamp2'].dt.dayofweek
book_train['Hour'] = book_train['TimeStamp2'].dt.hour
book_train['Minute'] = book_train['TimeStamp2'].dt.minute
book_train['Second'] = book_train['TimeStamp2'].dt.second
book_train.head()

,TimeStamp,Bid1,Bid1Size,Ask1,Ask1Size,Bid2,Bid2Size,Ask2,Ask2Size,Bid3,Bid3Size,Ask3,Ask3Size,Bid4,Bid4Size,Ask4,Ask4Size,Bid5,Bid5Size,Ask5,Ask5Size,midpt,nextMidpt,model,TimeStamp2,Week,Hour,Minute,Second
index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
137,1585704603505567,1113.00,1900,1115.03,100,1112.22,2800,1116.00,500,1112.00,600,1117.0,100,1111.21,3000,1117.99,8358,1111.11,1600,1118.0,1300,1114.015,1115.265,1,2020-04-01 01:30:03.505567,2,1,30,3
138,1585704606961840,1115.03,100,1115.50,100,1114.44,200,1117.00,940,1113.01,1000,1117.1,3200,1113.00,1900,1117.40,4000,1112.22,2800,1117.5,100,1115.265,1116.495,1,2020-04-01 01:30:06.961840,2,1,30,6
139,1585704609513298,1116.00,1600,1116.99,100,1115.03,300,1117.00,1240,1114.44,200,1117.1,3200,1113.01,600,1117.40,4000,1113.00,2000,1117.5,100,1116.495,1113.795,1,2020-04-01 01:30:09.513298,2,1,30,9
140,1585704612124381,1113.15,300,1114.44,400,1113.01,600,1116.99,100,1113.00,2000,1117.1,140,1112.22,2800,1117.40,4000,1112.00,1600,1117.5,100,1113.795,1111.565,1,2020-04-01 01:30:12.124381,2,1,30,12
141,1585704615123840,1111.51,100,1111.62,2200,1111.50,200,1116.00,100,1111.30,100,1116.5,100,1111.21,3000,1117.00,300,1111.18,200,1117.4,4000,1111.565,1114.200,1,2020-04-01 01:30:15.123840,2,1,30,15


In [8]:
book_train = book_train.reset_index()
book_train.head()

,index,TimeStamp,Bid1,Bid1Size,Ask1,Ask1Size,Bid2,Bid2Size,Ask2,Ask2Size,Bid3,Bid3Size,Ask3,Ask3Size,Bid4,Bid4Size,Ask4,Ask4Size,Bid5,Bid5Size,Ask5,Ask5Size,midpt,nextMidpt,model,TimeStamp2,Week,Hour,Minute,Second
0,137,1585704603505567,1113.00,1900,1115.03,100,1112.22,2800,1116.00,500,1112.00,600,1117.0,100,1111.21,3000,1117.99,8358,1111.11,1600,1118.0,1300,1114.015,1115.265,1,2020-04-01 01:30:03.505567,2,1,30,3
1,138,1585704606961840,1115.03,100,1115.50,100,1114.44,200,1117.00,940,1113.01,1000,1117.1,3200,1113.00,1900,1117.40,4000,1112.22,2800,1117.5,100,1115.265,1116.495,1,2020-04-01 01:30:06.961840,2,1,30,6
2,139,1585704609513298,1116.00,1600,1116.99,100,1115.03,300,1117.00,1240,1114.44,200,1117.1,3200,1113.01,600,1117.40,4000,1113.00,2000,1117.5,100,1116.495,1113.795,1,2020-04-01 01:30:09.513298,2,1,30,9
3,140,1585704612124381,1113.15,300,1114.44,400,1113.01,600,1116.99,100,1113.00,2000,1117.1,140,1112.22,2800,1117.40,4000,1112.00,1600,1117.5,100,1113.795,1111.565,1,2020-04-01 01:30:12.124381,2,1,30,12
4,141,1585704615123840,1111.51,100,1111.62,2200,1111.50,200,1116.00,100,1111.30,100,1116.5,100,1111.21,3000,1117.00,300,1111.18,200,1117.4,4000,1111.565,1114.200,1,2020-04-01 01:30:15.123840,2,1,30,15


In [9]:
%%time

col_bid_size = ['Bid1Size','Bid2Size','Bid3Size','Bid4Size','Bid5Size'] #買
col_ask_size = ['Ask1Size','Ask2Size','Ask3Size','Ask4Size','Ask5Size'] #賣

col_bid = ['Bid1','Bid2','Bid3','Bid4','Bid5'] #買
col_ask = ['Ask1','Ask2','Ask3','Ask4','Ask5'] #賣


## 動能(漲跌相對強度)
book_train['book_size_diff'] =  book_train[col_bid_size].sum(axis=1) - book_train[col_ask].sum(axis=1) #委買掛單總數-委賣掛單總數
book_train['book_size_ratio'] =  book_train[col_bid_size].sum(axis=1) / book_train[col_ask].sum(axis=1) #委買掛單總數/委賣掛單總數


## 基本描述統計值(最大、最小、平均)_ask
def get_avg1(row):
    avg = (row['Ask1Size']*row['Ask1']+row['Ask2Size']*row['Ask2']+row['Ask3Size']*row['Ask3']+row['Ask4Size']*row['Ask4']+row['Ask5Size']*row['Ask5']) / (row['Ask1Size']+row['Ask2Size']+row['Ask3Size']+row['Ask4Size']+row['Ask5Size'])
    return avg

book_train['pre_ask_min'] = book_train.shift(1)[col_ask].min(axis=1)
book_train['pre_ask_max'] = book_train.shift(1)[col_ask].max(axis=1)
book_train['pre_ask_avg'] = book_train.shift(1).apply(lambda row: get_avg1(row), axis=1)

## 基本描述統計值(眾數、四分位數)_ask 
def flatten_extend(matrix): #把巢狀lsit攤平
    flat_list = []
    for row in matrix:
        flat_list.extend(row)
    return flat_list

pre_book_ask_mode = []
pre_book_ask_q1 = []
pre_book_ask_q2 = []
pre_book_ask_q3 = []

for i in range(1, len(book_train)):
    term_data = []
    term_data = map( lambda x,y: book_train.loc[i-1,x].repeat(int(book_train.loc[i-1,y])) ,col_ask ,col_ask_size )
    term_data = list(term_data)
    term_data = flatten_extend(term_data)
    pre_book_ask_mode.append( mode(term_data) )
    pre_book_ask_q1.append( np.quantile(term_data,0.25) )
    pre_book_ask_q2.append( np.quantile(term_data,0.5) )
    pre_book_ask_q3.append( np.quantile(term_data,0.75) )

pre_book_ask_mode.insert( 0, pre_book_ask_mode[0] )
pre_book_ask_q1.insert( 0, pre_book_ask_q1[0] )
pre_book_ask_q2.insert( 0, pre_book_ask_q2[0] )
pre_book_ask_q3.insert( 0, pre_book_ask_q3[0] )

book_train['pre_book_ask_mode'] = pre_book_ask_mode
book_train['pre_book_ask_q1'] = pre_book_ask_q1
book_train['pre_book_ask_q2'] = pre_book_ask_q2
book_train['pre_book_ask_q3'] = pre_book_ask_q3

book_train.head()

,index,TimeStamp,Bid1,Bid1Size,Ask1,Ask1Size,Bid2,Bid2Size,Ask2,Ask2Size,Bid3,Bid3Size,Ask3,Ask3Size,Bid4,Bid4Size,Ask4,Ask4Size,Bid5,Bid5Size,Ask5,Ask5Size,midpt,nextMidpt,model,TimeStamp2,Week,Hour,Minute,Second,book_size_diff,book_size_ratio,pre_ask_min,pre_ask_max,pre_ask_avg,pre_book_ask_mode,pre_book_ask_q1,pre_book_ask_q2,pre_book_ask_q3
0,137,1585704603505567,1113.00,1900,1115.03,100,1112.22,2800,1116.00,500,1112.00,600,1117.0,100,1111.21,3000,1117.99,8358,1111.11,1600,1118.0,1300,1114.015,1115.265,1,2020-04-01 01:30:03.505567,2,1,30,3,4315.98,1.772916,NaN,NaN,NaN,1117.99,1117.99,1117.99,1117.99
1,138,1585704606961840,1115.03,100,1115.50,100,1114.44,200,1117.00,940,1113.01,1000,1117.1,3200,1113.00,1900,1117.40,4000,1112.22,2800,1117.5,100,1115.265,1116.495,1,2020-04-01 01:30:06.961840,2,1,30,6,415.50,1.074402,1115.03,1118.0,1117.857059,1117.99,1117.99,1117.99,1117.99
2,139,1585704609513298,1116.00,1600,1116.99,100,1115.03,300,1117.00,1240,1114.44,200,1117.1,3200,1113.01,600,1117.40,4000,1113.00,2000,1117.5,100,1116.495,1113.795,1,2020-04-01 01:30:09.513298,2,1,30,9,-885.99,0.841391,1115.50,1117.5,1117.218225,1117.40,1117.10,1117.10,1117.40
3,140,1585704612124381,1113.15,300,1114.44,400,1113.01,600,1116.99,100,1113.00,2000,1117.1,140,1112.22,2800,1117.40,4000,1112.00,1600,1117.5,100,1113.795,1111.565,1,2020-04-01 01:30:12.124381,2,1,30,12,1716.57,1.307440,1116.99,1117.5,1117.227894,1117.40,1117.10,1117.10,1117.40
4,141,1585704615123840,1111.51,100,1111.62,2200,1111.50,200,1116.00,100,1111.30,100,1116.5,100,1111.21,3000,1117.00,300,1111.18,200,1117.4,4000,1111.565,1114.200,1,2020-04-01 01:30:15.123840,2,1,30,15,-1978.52,0.645332,1114.44,1117.5,1117.134810,1117.40,1117.40,1117.40,1117.40


In [10]:
%%time

## 基本描述統計值(最大、最小、平均)_bid
def get_avg2(row):
    avg = (row['Bid1Size']*row['Bid1']+row['Bid2Size']*row['Bid2']+row['Bid3Size']*row['Bid3']+row['Bid4Size']*row['Bid4']+row['Bid5Size']*row['Bid5']) / (row['Bid1Size']+row['Bid2Size']+row['Bid3Size']+row['Bid4Size']+row['Bid5Size'])
    return avg

book_train['pre_bid_min'] = book_train.shift(1)[col_bid].min(axis=1)
book_train['pre_bid_max'] = book_train.shift(1)[col_bid].max(axis=1)
book_train['pre_bid_avg'] = book_train.shift(1).apply(lambda row: get_avg2(row), axis=1)

## 基本描述統計值(眾數、四分位數)_bid
pre_book_bid_mode = []
pre_book_bid_q1 = []
pre_book_bid_q2 = []
pre_book_bid_q3 = []

for i in range(1, len(book_train)):
    term_data = []
    term_data = map(lambda x,y: book_train.loc[i-1,x].repeat(int(book_train.loc[i-1,y])) ,col_bid ,col_bid_size )
    term_data = list(term_data)
    term_data = flatten_extend(term_data)
    pre_book_bid_mode.append( mode(term_data) )
    pre_book_bid_q1.append( np.quantile(term_data,0.25) )
    pre_book_bid_q2.append( np.quantile(term_data,0.5) )
    pre_book_bid_q3.append( np.quantile(term_data,0.75) )

pre_book_bid_mode.insert( 0, pre_book_bid_mode[0] )
pre_book_bid_q1.insert( 0, pre_book_bid_q1[0] )
pre_book_bid_q2.insert( 0, pre_book_bid_q2[0] )
pre_book_bid_q3.insert( 0, pre_book_bid_q3[0] )

book_train['pre_book_bid_mode'] = pre_book_bid_mode
book_train['pre_book_bid_q1'] = pre_book_bid_q1
book_train['pre_book_bid_q2'] = pre_book_bid_q2
book_train['pre_book_bid_q3'] = pre_book_bid_q3

book_train.head()

,index,TimeStamp,Bid1,Bid1Size,Ask1,Ask1Size,Bid2,Bid2Size,Ask2,Ask2Size,Bid3,Bid3Size,Ask3,Ask3Size,Bid4,Bid4Size,Ask4,Ask4Size,Bid5,Bid5Size,Ask5,Ask5Size,midpt,nextMidpt,model,TimeStamp2,Week,Hour,Minute,Second,book_size_diff,book_size_ratio,pre_ask_min,pre_ask_max,pre_ask_avg,pre_book_ask_mode,pre_book_ask_q1,pre_book_ask_q2,pre_book_ask_q3,pre_bid_min,pre_bid_max,pre_bid_avg,pre_book_bid_mode,pre_book_bid_q1,pre_book_bid_q2,pre_book_bid_q3
0,137,1585704603505567,1113.00,1900,1115.03,100,1112.22,2800,1116.00,500,1112.00,600,1117.0,100,1111.21,3000,1117.99,8358,1111.11,1600,1118.0,1300,1114.015,1115.265,1,2020-04-01 01:30:03.505567,2,1,30,3,4315.98,1.772916,NaN,NaN,NaN,1117.99,1117.99,1117.99,1117.99,NaN,NaN,NaN,1111.21,1111.21,1112.00,1112.22
1,138,1585704606961840,1115.03,100,1115.50,100,1114.44,200,1117.00,940,1113.01,1000,1117.1,3200,1113.00,1900,1117.40,4000,1112.22,2800,1117.5,100,1115.265,1116.495,1,2020-04-01 01:30:06.961840,2,1,30,6,415.50,1.074402,1115.03,1118.0,1117.857059,1117.99,1117.99,1117.99,1117.99,1111.11,1113.00,1111.870909,1111.21,1111.21,1112.00,1112.22
2,139,1585704609513298,1116.00,1600,1116.99,100,1115.03,300,1117.00,1240,1114.44,200,1117.1,3200,1113.01,600,1117.40,4000,1113.00,2000,1117.5,100,1116.495,1113.795,1,2020-04-01 01:30:09.513298,2,1,30,9,-885.99,0.841391,1115.50,1117.5,1117.218225,1117.40,1117.10,1117.10,1117.40,1112.22,1115.03,1112.719500,1112.22,1112.22,1113.00,1113.00
3,140,1585704612124381,1113.15,300,1114.44,400,1113.01,600,1116.99,100,1113.00,2000,1117.1,140,1112.22,2800,1117.40,4000,1112.00,1600,1117.5,100,1113.795,1111.565,1,2020-04-01 01:30:12.124381,2,1,30,12,1716.57,1.307440,1116.99,1117.5,1117.227894,1117.40,1117.10,1117.10,1117.40,1113.00,1116.00,1114.213404,1113.00,1113.00,1113.01,1116.00
4,141,1585704615123840,1111.51,100,1111.62,2200,1111.50,200,1116.00,100,1111.30,100,1116.5,100,1111.21,3000,1117.00,300,1111.18,200,1117.4,4000,1111.565,1114.200,1,2020-04-01 01:30:15.123840,2,1,30,15,-1978.52,0.645332,1114.44,1117.5,1117.134810,1117.40,1117.40,1117.40,1117.40,1112.00,1113.15,1112.488630,1112.22,1112.22,1112.22,1113.00


In [11]:
## 移動平均線
book_train['book_ma_6s'] =  book_train['midpt'].shift(1).rolling(1).mean()
book_train['book_ma_12s'] = book_train['midpt'].shift(1).rolling(2).mean()
book_train['book_ma_24s'] = book_train['midpt'].shift(1).rolling(3).mean()
book_train['book_ma_48s'] = book_train['midpt'].shift(1).rolling(4).mean()
book_train['book_ma_96s'] = book_train['midpt'].shift(1).rolling(5).mean()

book_train['book_mstd_12s'] = book_train['midpt'].shift(1).rolling(2).std()
book_train['book_mstd_24s'] = book_train['midpt'].shift(1).rolling(3).std()
book_train['book_mstd_48s'] = book_train['midpt'].shift(1).rolling(4).std()
book_train['book_mstd_96s'] = book_train['midpt'].shift(1).rolling(5).std()

book_train.head(10)

,index,TimeStamp,Bid1,Bid1Size,Ask1,Ask1Size,Bid2,Bid2Size,Ask2,Ask2Size,Bid3,Bid3Size,Ask3,Ask3Size,Bid4,Bid4Size,Ask4,Ask4Size,Bid5,Bid5Size,Ask5,Ask5Size,midpt,nextMidpt,model,TimeStamp2,Week,Hour,Minute,Second,book_size_diff,book_size_ratio,pre_ask_min,pre_ask_max,pre_ask_avg,pre_book_ask_mode,pre_book_ask_q1,pre_book_ask_q2,pre_book_ask_q3,pre_bid_min,pre_bid_max,pre_bid_avg,pre_book_bid_mode,pre_book_bid_q1,pre_book_bid_q2,pre_book_bid_q3,book_ma_6s,book_ma_12s,book_ma_24s,book_ma_48s,book_ma_96s,book_mstd_12s,book_mstd_24s,book_mstd_48s,book_mstd_96s
0,137,1585704603505567,1113.00,1900,1115.03,100,1112.22,2800,1116.00,500,1112.00,600,1117.00,100,1111.21,3000,1117.99,8358,1111.11,1600,1118.00,1300,1114.015,1115.265,1,2020-04-01 01:30:03.505567,2,1,30,3,4315.98,1.772916,NaN,NaN,NaN,1117.99,1117.9900,1117.990,1117.990,NaN,NaN,NaN,1111.21,1111.21,1112.00,1112.22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,138,1585704606961840,1115.03,100,1115.50,100,1114.44,200,1117.00,940,1113.01,1000,1117.10,3200,1113.00,1900,1117.40,4000,1112.22,2800,1117.50,100,1115.265,1116.495,1,2020-04-01 01:30:06.961840,2,1,30,6,415.50,1.074402,1115.03,1118.00,1117.857059,1117.99,1117.9900,1117.990,1117.990,1111.11,1113.00,1111.870909,1111.21,1111.21,1112.00,1112.22,1114.015,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,139,1585704609513298,1116.00,1600,1116.99,100,1115.03,300,1117.00,1240,1114.44,200,1117.10,3200,1113.01,600,1117.40,4000,1113.00,2000,1117.50,100,1116.495,1113.795,1,2020-04-01 01:30:09.513298,2,1,30,9,-885.99,0.841391,1115.50,1117.50,1117.218225,1117.40,1117.1000,1117.100,1117.400,1112.22,1115.03,1112.719500,1112.22,1112.22,1113.00,1113.00,1115.265,1114.6400,NaN,NaN,NaN,0.883883,NaN,NaN,NaN
3,140,1585704612124381,1113.15,300,1114.44,400,1113.01,600,1116.99,100,1113.00,2000,1117.10,140,1112.22,2800,1117.40,4000,1112.00,1600,1117.50,100,1113.795,1111.565,1,2020-04-01 01:30:12.124381,2,1,30,12,1716.57,1.307440,1116.99,1117.50,1117.227894,1117.40,1117.1000,1117.100,1117.400,1113.00,1116.00,1114.213404,1113.00,1113.00,1113.01,1116.00,1116.495,1115.8800,1115.258333,NaN,NaN,0.869741,1.240013,NaN,NaN
4,141,1585704615123840,1111.51,100,1111.62,2200,1111.50,200,1116.00,100,1111.30,100,1116.50,100,1111.21,3000,1117.00,300,1111.18,200,1117.40,4000,1111.565,1114.200,1,2020-04-01 01:30:15.123840,2,1,30,15,-1978.52,0.645332,1114.44,1117.50,1117.134810,1117.40,1117.4000,1117.400,1117.400,1112.00,1113.15,1112.488630,1112.22,1112.22,1112.22,1113.00,1113.795,1115.1450,1115.185000,1114.89250,NaN,1.909188,1.351777,1.249170,NaN
5,142,1585704618121704,1111.00,2200,1117.40,4100,1110.90,2100,1117.50,100,1110.02,100,1117.51,100,1110.01,200,1117.60,15000,1110.00,2000,1117.99,7758,1114.200,1111.895,1,2020-04-01 01:30:18.121704,2,1,30,18,1012.00,1.181102,1111.62,1117.40,1115.449851,1117.40,1111.6200,1117.400,1117.400,1111.18,1111.51,1111.235278,1111.21,1111.21,1111.21,1111.21,1111.565,1112.6800,1113.951667,1114.28000,1114.227,1.576848,2.468731,2.119976,1.839774
6,143,1585704621132090,1111.51,100,1112.28,100,1111.00,2700,1115.50,100,1110.90,2100,1115.76,200,1110.02,100,1116.79,100,1110.01,200,1117.39,1100,1111.895,1111.115,1,2020-04-01 01:30:21.132090,2,1,30,21,-377.72,0.932281,1117.40,1117.99,1117.680812,1117.60,1117.6000,1117.600,1117.990,1110.00,1111.00,1110.620303,1111.00,1110.00,1110.90,1111.00,1114.200,1112.8825,1113.186667,1114.01375,1114.264,1.863226,1.418929,2.019531,1.836302
7,144,1585704624123763,1111.11,300,1111.12,1000,1111.00,2700,1112.28,200,1110.90,2100,1115.50,200,1110.02,100,1115.76,200,1110.01,200,1116.79,100,1111.115,1113.890,1,2020-04-01 01:30:24.123763,2,1,30,24,-171.45,0.969227,1112.28,1117.39,1116.711250,1117.39,1116.5325,1117.390,1117.390,1110.01,1111.51,1110.912500,1111.00,1110.90,1111.00,1111.00,1111.895,1113.0475,1112.553333,1112.86375,1113.590,1.629881,1.435569,1.326401,1.989146
8,145,1585704627126163,1112.28,2900,1115.50,200,1111.12,100,1115.76,200,1111.11,300,1116.49,200,1111.03,500,1116.79,100,1111.01,100,1117.30,100,1113.890,1113.640,1,2020-04-01 

In [12]:
# 加上'ts_id'作為和trade data mapping用
book_train['ts_id'] = range(0,len(book_train),1)

In [13]:
book_train.head()

,index,TimeStamp,Bid1,Bid1Size,Ask1,Ask1Size,Bid2,Bid2Size,Ask2,Ask2Size,Bid3,Bid3Size,Ask3,Ask3Size,Bid4,Bid4Size,Ask4,Ask4Size,Bid5,Bid5Size,Ask5,Ask5Size,midpt,nextMidpt,model,TimeStamp2,Week,Hour,Minute,Second,book_size_diff,book_size_ratio,pre_ask_min,pre_ask_max,pre_ask_avg,pre_book_ask_mode,pre_book_ask_q1,pre_book_ask_q2,pre_book_ask_q3,pre_bid_min,pre_bid_max,pre_bid_avg,pre_book_bid_mode,pre_book_bid_q1,pre_book_bid_q2,pre_book_bid_q3,book_ma_6s,book_ma_12s,book_ma_24s,book_ma_48s,book_ma_96s,book_mstd_12s,book_mstd_24s,book_mstd_48s,book_mstd_96s,ts_id
0,137,1585704603505567,1113.00,1900,1115.03,100,1112.22,2800,1116.00,500,1112.00,600,1117.0,100,1111.21,3000,1117.99,8358,1111.11,1600,1118.0,1300,1114.015,1115.265,1,2020-04-01 01:30:03.505567,2,1,30,3,4315.98,1.772916,NaN,NaN,NaN,1117.99,1117.99,1117.99,1117.99,NaN,NaN,NaN,1111.21,1111.21,1112.00,1112.22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,138,1585704606961840,1115.03,100,1115.50,100,1114.44,200,1117.00,940,1113.01,1000,1117.1,3200,1113.00,1900,1117.40,4000,1112.22,2800,1117.5,100,1115.265,1116.495,1,2020-04-01 01:30:06.961840,2,1,30,6,415.50,1.074402,1115.03,1118.0,1117.857059,1117.99,1117.99,1117.99,1117.99,1111.11,1113.00,1111.870909,1111.21,1111.21,1112.00,1112.22,1114.015,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,139,1585704609513298,1116.00,1600,1116.99,100,1115.03,300,1117.00,1240,1114.44,200,1117.1,3200,1113.01,600,1117.40,4000,1113.00,2000,1117.5,100,1116.495,1113.795,1,2020-04-01 01:30:09.513298,2,1,30,9,-885.99,0.841391,1115.50,1117.5,1117.218225,1117.40,1117.10,1117.10,1117.40,1112.22,1115.03,1112.719500,1112.22,1112.22,1113.00,1113.00,1115.265,1114.640,NaN,NaN,NaN,0.883883,NaN,NaN,NaN,2
3,140,1585704612124381,1113.15,300,1114.44,400,1113.01,600,1116.99,100,1113.00,2000,1117.1,140,1112.22,2800,1117.40,4000,1112.00,1600,1117.5,100,1113.795,1111.565,1,2020-04-01 01:30:12.124381,2,1,30,12,1716.57,1.307440,1116.99,1117.5,1117.227894,1117.40,1117.10,1117.10,1117.40,1113.00,1116.00,1114.213404,1113.00,1113.00,1113.01,1116.00,1116.495,1115.880,1115.258333,NaN,NaN,0.869741,1.240013,NaN,NaN,3
4,141,1585704615123840,1111.51,100,1111.62,2200,1111.50,200,1116.00,100,1111.30,100,1116.5,100,1111.21,3000,1117.00,300,1111.18,200,1117.4,4000,1111.565,1114.200,1,2020-04-01 01:30:15.123840,2,1,30,15,-1978.52,0.645332,1114.44,1117.5,1117.134810,1117.40,1117.40,1117.40,1117.40,1112.00,1113.15,1112.488630,1112.22,1112.22,1112.22,1113.00,1113.795,1115.145,1115.185000,1114.8925,NaN,1.909188,1.351777,1.24917,NaN,4


In [14]:
## 交易量(特徵) -----------------------------------------------------------------------------------------
raw_trade_train = pd.read_parquet('trade/600519/20200401.parquet.gzip')
trade_train = raw_trade_train.copy()

In [15]:
trade_train['TimeStamp2'] = pd.to_datetime(trade_train['TimeStamp'].values, unit='us')
trade_train.head()

,TimeStamp,BidID,AskID,BuyVolume,BuyPrice,SellVolume,SellPrice,TimeStamp2
index,,,,,,,,
213,1585704600549504,99924,141932,0,0.0,100,1115.5,2020-04-01 01:30:00.549504
214,1585704600551309,142138,141795,100,1116.0,0,0.0,2020-04-01 01:30:00.551309
215,1585704601244086,86301,142421,0,0.0,100,1115.0,2020-04-01 01:30:01.244086
216,1585704601252795,141888,143205,0,0.0,100,1115.0,2020-04-01 01:30:01.252795
217,1585704601256878,141888,143923,0,0.0,200,1115.0,2020-04-01 01:30:01.256878


In [16]:
## 時間
trade_train['Week'] = trade_train['TimeStamp2'].dt.dayofweek
trade_train['Hour'] = trade_train['TimeStamp2'].dt.hour
trade_train['Minute'] = trade_train['TimeStamp2'].dt.minute
trade_train['Second'] = trade_train['TimeStamp2'].dt.second
trade_train.head()

,TimeStamp,BidID,AskID,BuyVolume,BuyPrice,SellVolume,SellPrice,TimeStamp2,Week,Hour,Minute,Second
index,,,,,,,,,,,,
213,1585704600549504,99924,141932,0,0.0,100,1115.5,2020-04-01 01:30:00.549504,2,1,30,0
214,1585704600551309,142138,141795,100,1116.0,0,0.0,2020-04-01 01:30:00.551309,2,1,30,0
215,1585704601244086,86301,142421,0,0.0,100,1115.0,2020-04-01 01:30:01.244086,2,1,30,1
216,1585704601252795,141888,143205,0,0.0,100,1115.0,2020-04-01 01:30:01.252795,2,1,30,1
217,1585704601256878,141888,143923,0,0.0,200,1115.0,2020-04-01 01:30:01.256878,2,1,30,1


In [17]:
# 交易量對應五檔報價"ts_id"

tstotsid_ = dict(zip(book_train['TimeStamp'].values, book_train['ts_id'].values))
# tstotsid_
# tstotsid_[1585704621132090]

import bisect
book_ts = book_train['TimeStamp'].values.tolist()  #dataframe to list

def get_ts_id(ts):  #ts: timestamp
    index_to_insert = bisect.bisect_left(book_ts, ts)  #回傳插入的位置，以0開頭
    try:
        return tstotsid_[book_ts[index_to_insert]]  #回傳對應的ts_id
    except:
        return -1
    
trade_train['ts_id'] = trade_train['TimeStamp'].apply(lambda ts: get_ts_id(ts))  #ts:trade每一列
trade_train.query('ts_id == -1')

,TimeStamp,BidID,AskID,BuyVolume,BuyPrice,SellVolume,SellPrice,TimeStamp2,Week,Hour,Minute,Second,ts_id
index,,,,,,,,,,,,,


In [18]:
# list_ = [10003, 10006, 10009, 10012, 10015, 10018]
# #index  0       1     2        3    4      5

# number = 10005
# index_to_insert = bisect.bisect_left(list_, number)
# print(f"need to insert {number} at {index_to_insert}")
# print(f"value at index {index_to_insert} is {list_[index_to_insert]}")
# # index = 1, insert 10006
# list_.insert(index_to_insert, number)
# list_

In [19]:
# 交易買賣差、交易買賣比
dict_tmp = (trade_train.groupby('ts_id')['SellVolume'].sum() - trade_train.groupby('ts_id')['BuyVolume'].sum()).to_dict()
trade_train['trade_size_diff'] = trade_train['ts_id'].map(dict_tmp)

dict_tmp = (trade_train.groupby('ts_id')['SellVolume'].sum() / trade_train.groupby('ts_id')['BuyVolume'].sum()).to_dict()
trade_train['trade_size_ratio'] = trade_train['ts_id'].map(dict_tmp)

trade_train.head()

,TimeStamp,BidID,AskID,BuyVolume,BuyPrice,SellVolume,SellPrice,TimeStamp2,Week,Hour,Minute,Second,ts_id,trade_size_diff,trade_size_ratio
index,,,,,,,,,,,,,,,
213,1585704600549504,99924,141932,0,0.0,100,1115.5,2020-04-01 01:30:00.549504,2,1,30,0,0,2500,5.166667
214,1585704600551309,142138,141795,100,1116.0,0,0.0,2020-04-01 01:30:00.551309,2,1,30,0,0,2500,5.166667
215,1585704601244086,86301,142421,0,0.0,100,1115.0,2020-04-01 01:30:01.244086,2,1,30,1,0,2500,5.166667
216,1585704601252795,141888,143205,0,0.0,100,1115.0,2020-04-01 01:30:01.252795,2,1,30,1,0,2500,5.166667
217,1585704601256878,141888,143923,0,0.0,200,1115.0,2020-04-01 01:30:01.256878,2,1,30,1,0,2500,5.166667


In [20]:
# 買、賣 各成交總次數
df_result = trade_train.groupby(trade_train['ts_id'])['BuyVolume'].agg(lambda x: x[x>0].count()).reset_index()
df_result = df_result.rename(columns={'BuyVolume':'trade_buy_cnt'})
trade_train = trade_train.merge(df_result, how='inner', on='ts_id')

df_result = trade_train.groupby(trade_train['ts_id'])['SellVolume'].agg(lambda x: x[x>0].count()).reset_index()
df_result = df_result.rename(columns={'SellVolume': 'trade_sell_cnt'})
trade_train = trade_train.merge(df_result, how='inner', on='ts_id')

trade_train['trade_cnt_diff'] = trade_train['trade_sell_cnt'] - trade_train['trade_buy_cnt'] #買成 > 賣成，預期價格跌。
trade_train.head()

,TimeStamp,BidID,AskID,BuyVolume,BuyPrice,SellVolume,SellPrice,TimeStamp2,Week,Hour,Minute,Second,ts_id,trade_size_diff,trade_size_ratio,trade_buy_cnt,trade_sell_cnt,trade_cnt_diff
0,1585704600549504,99924,141932,0,0.0,100,1115.5,2020-04-01 01:30:00.549504,2,1,30,0,0,2500,5.166667,6,8,2
1,1585704600551309,142138,141795,100,1116.0,0,0.0,2020-04-01 01:30:00.551309,2,1,30,0,0,2500,5.166667,6,8,2
2,1585704601244086,86301,142421,0,0.0,100,1115.0,2020-04-01 01:30:01.244086,2,1,30,1,0,2500,5.166667,6,8,2
3,1585704601252795,141888,143205,0,0.0,100,1115.0,2020-04-01 01:30:01.252795,2,1,30,1,0,2500,5.166667,6,8,2
4,1585704601256878,141888,143923,0,0.0,200,1115.0,2020-04-01 01:30:01.256878,2,1,30,1,0,2500,5.166667,6,8,2


In [21]:
# 內/外盤成交價(sell,buy)移動平均線、移動標準差
#sell
df_result = trade_train.query('SellPrice > 0').groupby('ts_id')['SellPrice'].rolling(1).mean().reset_index()
df_result = df_result.drop_duplicates('ts_id', keep='last').drop(columns=['level_1'])
df_result = df_result.rename(columns={'SellPrice':'sell_ma1'})
trade_train = trade_train.merge(df_result, how='inner', on='ts_id')

df_result = trade_train.query('SellPrice > 0').groupby('ts_id')['SellPrice'].rolling(3).mean().reset_index()
df_result = df_result.drop_duplicates('ts_id', keep='last').drop(columns=['level_1'])
df_result = df_result.rename(columns={'SellPrice':'sell_ma3'})
trade_train = trade_train.merge(df_result, how='inner', on='ts_id')

df_result = trade_train.query('SellPrice > 0').groupby('ts_id')['SellPrice'].rolling(6).mean().reset_index()
df_result = df_result.drop_duplicates('ts_id', keep='last').drop(columns=['level_1'])
df_result = df_result.rename(columns={'SellPrice':'sell_ma6'})
trade_train = trade_train.merge(df_result, how='inner', on='ts_id')

df_result = trade_train.query('SellPrice > 0').groupby('ts_id')['SellPrice'].rolling(3).std().reset_index()
df_result = df_result.drop_duplicates('ts_id', keep='last').drop(columns=['level_1'])
df_result = df_result.rename(columns={'SellPrice':'sell_mstd3'})
trade_train = trade_train.merge(df_result, how='inner', on='ts_id')

df_result = trade_train.query('SellPrice > 0').groupby('ts_id')['SellPrice'].rolling(6).std().reset_index()
df_result = df_result.drop_duplicates('ts_id', keep='last').drop(columns=['level_1'])
df_result = df_result.rename(columns={'SellPrice':'sell_mstd6'})
trade_train = trade_train.merge(df_result, how='inner', on='ts_id')

#buy
df_result = trade_train.query('BuyPrice > 0').groupby('ts_id')['BuyPrice'].rolling(1).mean().reset_index()
df_result = df_result.drop_duplicates('ts_id', keep='last').drop(columns=['level_1'])
df_result = df_result.rename(columns={'BuyPrice':'buyl_ma1'})
trade_train = trade_train.merge(df_result, how='inner', on='ts_id')

df_result = trade_train.query('BuyPrice > 0').groupby('ts_id')['BuyPrice'].rolling(3).mean().reset_index()
df_result = df_result.drop_duplicates('ts_id', keep='last').drop(columns=['level_1'])
df_result = df_result.rename(columns={'BuyPrice':'buyl_ma3'})
trade_train = trade_train.merge(df_result, how='inner', on='ts_id')

df_result = trade_train.query('BuyPrice > 0').groupby('ts_id')['BuyPrice'].rolling(6).mean().reset_index()
df_result = df_result.drop_duplicates('ts_id', keep='last').drop(columns=['level_1'])
df_result = df_result.rename(columns={'BuyPrice':'buyl_ma6'})
trade_train = trade_train.merge(df_result, how='inner', on='ts_id')

df_result = trade_train.query('BuyPrice > 0').groupby('ts_id')['BuyPrice'].rolling(3).std().reset_index()
df_result = df_result.drop_duplicates('ts_id', keep='last').drop(columns=['level_1'])
df_result = df_result.rename(columns={'BuyPrice':'buyl_mstd3'})
trade_train = trade_train.merge(df_result, how='inner', on='ts_id')

df_result = trade_train.query('BuyPrice > 0').groupby('ts_id')['BuyPrice'].rolling(6).std().reset_index()
df_result = df_result.drop_duplicates('ts_id', keep='last').drop(columns=['level_1'])
df_result = df_result.rename(columns={'BuyPrice':'buyl_mstd6'})
trade_train = trade_train.merge(df_result, how='inner', on='ts_id')

trade_train.head()

,TimeStamp,BidID,AskID,BuyVolume,BuyPrice,SellVolume,SellPrice,TimeStamp2,Week,Hour,Minute,Second,ts_id,trade_size_diff,trade_size_ratio,trade_buy_cnt,trade_sell_cnt,trade_cnt_diff,sell_ma1,sell_ma3,sell_ma6,sell_mstd3,sell_mstd6,buyl_ma1,buyl_ma3,buyl_ma6,buyl_mstd3,buyl_mstd6
0,1585704600549504,99924,141932,0,0.0,100,1115.5,2020-04-01 01:30:00.549504,2,1,30,0,0,2500,5.166667,6,8,2,1113.01,1114.336667,1114.668333,1.148927,0.812414,1115.99,1115.74,1115.703333,0.245153,0.242872
1,1585704600551309,142138,141795,100,1116.0,0,0.0,2020-04-01 01:30:00.551309,2,1,30,0,0,2500,5.166667,6,8,2,1113.01,1114.336667,1114.668333,1.148927,0.812414,1115.99,1115.74,1115.703333,0.245153,0.242872
2,1585704601244086,86301,142421,0,0.0,100,1115.0,2020-04-01 01:30:01.244086,2,1,30,1,0,2500,5.166667,6,8,2,1113.01,1114.336667,1114.668333,1.148927,0.812414,1115.99,1115.74,1115.703333,0.245153,0.242872
3,1585704601252795,141888,143205,0,0.0,100,1115.0,2020-04-01 01:30:01.252795,2,1,30,1,0,2500,5.166667,6,8,2,1113.01,1114.336667,1114.668333,1.148927,0.812414,1115.99,1115.74,1115.703333,0.245153,0.242872
4,1585704601256878,141888,143923,0,0.0,200,1115.0,2020-04-01 01:30:01.256878,2,1,30,1,0,2500,5.166667,6,8,2,1113.01,1114.336667,1114.668333,1.148927,0.812414,1115.99,1115.74,1115.703333,0.245153,0.242872


In [22]:
# ts2buyprice = {}

# for i, row in trade_train.iterrows():  #iterrows
#     ts = row['ts_id']
#     buyvolume = row['BuyVolume']
#     buyprice = row['BuyPrice']
#     if ts not in ts2buyprice:
#         ts2buyprice[ts] = []
#     for j in range(buyvolume // 100):
#         ts2buyprice[ts].append(buyprice)

# ts2buypriceq25 = {}
# for key, value in ts2buyprice.items():
#     value = sorted(value)
#     if (len(value) > 0):
#         ts2buypriceq25[key] = np.percentile(np.array(value), 25)

In [24]:
# 描述性統計: 前一個時段所有內/外盤成交的 (最大、最小、平均數、四分位數、眾數)
# %%time

#最大、最小
dict_tmp = trade_train.groupby('ts_id')['BuyPrice'].max().to_dict()
trade_train['buy_max'] = trade_train['ts_id'].map(dict_tmp)

dict_tmp = trade_train.groupby('ts_id')['BuyPrice'].agg(lambda x: x[x>0].min()).to_dict()
trade_train['buy_min'] = trade_train['ts_id'].map(dict_tmp)

#平均數、四分位數、眾數
pre_buy_q1 = {}
pre_buy_q2 = {}
pre_buy_q3 = {}
pre_buy_avg = {}
pre_buy_mode = {}

for tsid in trade_train['ts_id'].unique():
    split_data = trade_train.query(f'ts_id == {tsid}')
    split_data = split_data.query('BuyVolume > 0').reset_index()
    
    term_data2=[]
    data_q1=[]
    data_q2=[]
    data_q3=[]
    data_avg=[]
    data_mode=[]
    
    for i in range(0,len(split_data)):
        term_data1 = []
        term_data1 = split_data.loc[i,'BuyPrice'].repeat(int(split_data.loc[i,'BuyVolume']//100))
        term_data1 = term_data1.tolist()
        term_data2 += term_data1
    
    if len(term_data2) > 0:
        data_q1 = np.quantile(term_data2, 0.25)
        data_q2 = np.quantile(term_data2, 0.5)
        data_q3 = np.quantile(term_data2, 0.75)
        data_avg = np.mean(term_data2)
        data_mode = mode(term_data2)
              
        pre_buy_q1[tsid] = data_q1
        pre_buy_q2[tsid] = data_q2
        pre_buy_q3[tsid] = data_q3
        pre_buy_avg[tsid] = data_avg
        pre_buy_mode[tsid] = data_mode
    
    else: 
        pre_buy_q1[tsid] = None
        pre_buy_q2[tsid] = None
        pre_buy_q3[tsid] = None
        pre_buy_avg[tsid] = None
        pre_buy_mode[tsid] = None
        
trade_train['pre_buy_q1'] = trade_train['ts_id'].map(pre_buy_q1)
trade_train['pre_buy_q2'] = trade_train['ts_id'].map(pre_buy_q2)
trade_train['pre_buy_q3'] = trade_train['ts_id'].map(pre_buy_q3)
trade_train['pre_buy_avg'] = trade_train['ts_id'].map(pre_buy_avg)
trade_train['pre_buy_mode'] = trade_train['ts_id'].map(pre_buy_mode)

trade_train.head()

,TimeStamp,BidID,AskID,BuyVolume,BuyPrice,SellVolume,SellPrice,TimeStamp2,Week,Hour,Minute,Second,ts_id,trade_size_diff,trade_size_ratio,trade_buy_cnt,trade_sell_cnt,trade_cnt_diff,sell_ma1,sell_ma3,sell_ma6,sell_mstd3,sell_mstd6,buyl_ma1,buyl_ma3,buyl_ma6,buyl_mstd3,buyl_mstd6,buy_max,buy_min,pre_buy_q1,pre_buy_q2,pre_buy_q3,pre_buy_avg,pre_buy_mode
0,1585704600549504,99924,141932,0,0.0,100,1115.5,2020-04-01 01:30:00.549504,2,1,30,0,0,2500,5.166667,6,8,2,1113.01,1114.336667,1114.668333,1.148927,0.812414,1115.99,1115.74,1115.703333,0.245153,0.242872,1116.0,1115.5,1115.5,1115.615,1115.925,1115.703333,1115.5
1,1585704600551309,142138,141795,100,1116.0,0,0.0,2020-04-01 01:30:00.551309,2,1,30,0,0,2500,5.166667,6,8,2,1113.01,1114.336667,1114.668333,1.148927,0.812414,1115.99,1115.74,1115.703333,0.245153,0.242872,1116.0,1115.5,1115.5,1115.615,1115.925,1115.703333,1115.5
2,1585704601244086,86301,142421,0,0.0,100,1115.0,2020-04-01 01:30:01.244086,2,1,30,1,0,2500,5.166667,6,8,2,1113.01,1114.336667,1114.668333,1.148927,0.812414,1115.99,1115.74,1115.703333,0.245153,0.242872,1116.0,1115.5,1115.5,1115.615,1115.925,1115.703333,1115.5
3,1585704601252795,141888,143205,0,0.0,100,1115.0,2020-04-01 01:30:01.252795,2,1,30,1,0,2500,5.166667,6,8,2,1113.01,1114.336667,1114.668333,1.148927,0.812414,1115.99,1115.74,1115.703333,0.245153,0.242872,1116.0,1115.5,1115.5,1115.615,1115.925,1115.703333,1115.5
4,1585704601256878,141888,143923,0,0.0,200,1115.0,2020-04-01 01:30:01.256878,2,1,30,1,0,2500,5.166667,6,8,2,1113.01,1114.336667,1114.668333,1.148927,0.812414,1115.99,1115.74,1115.703333,0.245153,0.242872,1116.0,1115.5,1115.5,1115.615,1115.925,1115.703333,1115.5


In [25]:
# 描述性統計: 前一個時段所有內/外盤成交的 (最大、最小、平均數、四分位數、眾數)
#%%time

# 最大、最小
dict_tmp = trade_train.groupby('ts_id')['SellPrice'].max().to_dict()
trade_train['sell_max'] = trade_train['ts_id'].map(dict_tmp)

dict_tmp = trade_train.groupby('ts_id')['SellPrice'].agg(lambda x: x[x>0].min()).to_dict()
trade_train['sell_min'] = trade_train['ts_id'].map(dict_tmp)

# 平均數、四分位數、眾數
pre_sell_q1 = {}
pre_sell_q2 = {}
pre_sell_q3 = {}
pre_sell_avg = {}
pre_sell_mode = {}

for tsid in trade_train['ts_id'].unique():
    split_data = trade_train.query(f'ts_id == {tsid}')
    split_data = split_data.query('SellVolume > 0').reset_index()
    
    term_data2=[]
    data_q1=[]
    data_q2=[]
    data_q3=[]
    data_avg=[]
    data_mode=[]
    
    for i in range(0,len(split_data)):
        term_data1 = []
        term_data1 = split_data.loc[i,'SellPrice'].repeat(int(split_data.loc[i,'SellVolume']//100))
        term_data1 = term_data1.tolist()
        term_data2 += term_data1
    
    if len(term_data2) > 0:
        data_q1 = np.quantile(term_data2, 0.25)
        data_q2 = np.quantile(term_data2, 0.5)
        data_q3 = np.quantile(term_data2, 0.75)
        data_avg = np.mean(term_data2)
        data_mode = mode(term_data2)
              
        pre_sell_q1[tsid] = data_q1
        pre_sell_q2[tsid] = data_q2
        pre_sell_q3[tsid] = data_q3
        pre_sell_avg[tsid] = data_avg
        pre_sell_mode[tsid] = data_mode
    
    else: 
        pre_sell_q1[tsid] = None
        pre_sell_q2[tsid] = None
        pre_sell_q3[tsid] = None
        pre_sell_avg[tsid] = None
        pre_sell_mode[tsid] = None
        
trade_train['pre_sell_q1'] = trade_train['ts_id'].map(pre_sell_q1)
trade_train['pre_sell_q2'] = trade_train['ts_id'].map(pre_sell_q2)
trade_train['pre_sell_q3'] = trade_train['ts_id'].map(pre_sell_q3)
trade_train['pre_sell_avg'] = trade_train['ts_id'].map(pre_sell_avg)
trade_train['pre_sell_mode'] = trade_train['ts_id'].map(pre_sell_mode)

trade_train.head()

,TimeStamp,BidID,AskID,BuyVolume,BuyPrice,SellVolume,SellPrice,TimeStamp2,Week,Hour,Minute,Second,ts_id,trade_size_diff,trade_size_ratio,trade_buy_cnt,trade_sell_cnt,trade_cnt_diff,sell_ma1,sell_ma3,sell_ma6,sell_mstd3,sell_mstd6,buyl_ma1,buyl_ma3,buyl_ma6,buyl_mstd3,buyl_mstd6,buy_max,buy_min,pre_buy_q1,pre_buy_q2,pre_buy_q3,pre_buy_avg,pre_buy_mode,sell_max,sell_min,pre_sell_q1,pre_sell_q2,pre_sell_q3,pre_sell_avg,pre_sell_mode
0,1585704600549504,99924,141932,0,0.0,100,1115.5,2020-04-01 01:30:00.549504,2,1,30,0,0,2500,5.166667,6,8,2,1113.01,1114.336667,1114.668333,1.148927,0.812414,1115.99,1115.74,1115.703333,0.245153,0.242872,1116.0,1115.5,1115.5,1115.615,1115.925,1115.703333,1115.5,1115.5,1113.01,1115.0,1115.0,1115.0,1114.951935,1115.0
1,1585704600551309,142138,141795,100,1116.0,0,0.0,2020-04-01 01:30:00.551309,2,1,30,0,0,2500,5.166667,6,8,2,1113.01,1114.336667,1114.668333,1.148927,0.812414,1115.99,1115.74,1115.703333,0.245153,0.242872,1116.0,1115.5,1115.5,1115.615,1115.925,1115.703333,1115.5,1115.5,1113.01,1115.0,1115.0,1115.0,1114.951935,1115.0
2,1585704601244086,86301,142421,0,0.0,100,1115.0,2020-04-01 01:30:01.244086,2,1,30,1,0,2500,5.166667,6,8,2,1113.01,1114.336667,1114.668333,1.148927,0.812414,1115.99,1115.74,1115.703333,0.245153,0.242872,1116.0,1115.5,1115.5,1115.615,1115.925,1115.703333,1115.5,1115.5,1113.01,1115.0,1115.0,1115.0,1114.951935,1115.0
3,1585704601252795,141888,143205,0,0.0,100,1115.0,2020-04-01 01:30:01.252795,2,1,30,1,0,2500,5.166667,6,8,2,1113.01,1114.336667,1114.668333,1.148927,0.812414,1115.99,1115.74,1115.703333,0.245153,0.242872,1116.0,1115.5,1115.5,1115.615,1115.925,1115.703333,1115.5,1115.5,1113.01,1115.0,1115.0,1115.0,1114.951935,1115.0
4,1585704601256878,141888,143923,0,0.0,200,1115.0,2020-04-01 01:30:01.256878,2,1,30,1,0,2500,5.166667,6,8,2,1113.01,1114.336667,1114.668333,1.148927,0.812414,1115.99,1115.74,1115.703333,0.245153,0.242872,1116.0,1115.5,1115.5,1115.615,1115.925,1115.703333,1115.5,1115.5,1113.01,1115.0,1115.0,1115.0,1114.951935,1115.0


In [26]:
#將五檔報價資料 和 成交量 資料結合，產生feature_data
feature_data = book_train.copy()

col_position = [i for i in range(12,41,1)]
col_position
trade_train_merge = trade_train.iloc[:, col_position].drop_duplicates()

feature_data = feature_data.merge(trade_train_merge, how='left', on='ts_id')

feature_data.head()

,index,TimeStamp,Bid1,Bid1Size,Ask1,Ask1Size,Bid2,Bid2Size,Ask2,Ask2Size,Bid3,Bid3Size,Ask3,Ask3Size,Bid4,Bid4Size,Ask4,Ask4Size,Bid5,Bid5Size,Ask5,Ask5Size,midpt,nextMidpt,model,TimeStamp2,Week,Hour,Minute,Second,book_size_diff,book_size_ratio,pre_ask_min,pre_ask_max,pre_ask_avg,pre_book_ask_mode,pre_book_ask_q1,pre_book_ask_q2,pre_book_ask_q3,pre_bid_min,pre_bid_max,pre_bid_avg,pre_book_bid_mode,pre_book_bid_q1,pre_book_bid_q2,pre_book_bid_q3,book_ma_6s,book_ma_12s,book_ma_24s,book_ma_48s,book_ma_96s,book_mstd_12s,book_mstd_24s,book_mstd_48s,book_mstd_96s,ts_id,trade_size_diff,trade_size_ratio,trade_buy_cnt,trade_sell_cnt,trade_cnt_diff,sell_ma1,sell_ma3,sell_ma6,sell_mstd3,sell_mstd6,buyl_ma1,buyl_ma3,buyl_ma6,buyl_mstd3,buyl_mstd6,buy_max,buy_min,pre_buy_q1,pre_buy_q2,pre_buy_q3,pre_buy_avg,pre_buy_mode,sell_max,sell_min,pre_sell_q1,pre_sell_q2,pre_sell_q3,pre_sell_avg
0,137,1585704603505567,1113.00,1900,1115.03,100,1112.22,2800,1116.00,500,1112.00,600,1117.0,100,1111.21,3000,1117.99,8358,1111.11,1600,1118.0,1300,1114.015,1115.265,1,2020-04-01 01:30:03.505567,2,1,30,3,4315.98,1.772916,NaN,NaN,NaN,1117.99,1117.99,1117.99,1117.99,NaN,NaN,NaN,1111.21,1111.21,1112.00,1112.22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,2500.0,5.166667,6.0,8.0,2.0,1113.01,1114.336667,1114.668333,1.148927,0.812414,1115.99,1115.740000,1115.703333,0.245153,0.242872,1116.00,1115.50,1115.50,1115.615,1115.9250,1115.703333,1115.50,1115.50,1113.01,1115.00,1115.00,1115.00,1114.951935
1,138,1585704606961840,1115.03,100,1115.50,100,1114.44,200,1117.00,940,1113.01,1000,1117.1,3200,1113.00,1900,1117.40,4000,1112.22,2800,1117.5,100,1115.265,1116.495,1,2020-04-01 01:30:06.961840,2,1,30,6,415.50,1.074402,1115.03,1118.0,1117.857059,1117.99,1117.99,1117.99,1117.99,1111.11,1113.00,1111.870909,1111.21,1111.21,1112.00,1112.22,1114.015,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,-3000.0,0.032258,19.0,1.0,-18.0,1113.11,NaN,NaN,NaN,NaN,1115.50,1115.500000,1115.725000,0.000000,0.551135,1116.85,1113.01,1113.01,1115.490,1115.5000,1114.798065,1113.01,1113.11,1113.11,1113.11,1113.11,1113.11,1113.110000
2,139,1585704609513298,1116.00,1600,1116.99,100,1115.03,300,1117.00,1240,1114.44,200,1117.1,3200,1113.01,600,1117.40,4000,1113.00,2000,1117.5,100,1116.495,1113.795,1,2020-04-01 01:30:09.513298,2,1,30,9,-885.99,0.841391,1115.50,1117.5,1117.218225,1117.40,1117.10,1117.10,1117.40,1112.22,1115.03,1112.719500,1112.22,1112.22,1113.00,1113.00,1115.265,1114.640,NaN,NaN,NaN,0.883883,NaN,NaN,NaN,2,-2200.0,0.083333,19.0,2.0,-17.0,1115.00,NaN,NaN,NaN,NaN,1117.00,1116.993333,1116.991667,0.005774,0.004082,1117.00,1115.40,1116.85,1116.850,1116.9300,1116.648182,1116.85,1115.00,1113.00,1113.50,1114.00,1114.50,1114.000000
3,140,1585704612124381,1113.15,300,1114.44,400,1113.01,600,1116.99,100,1113.00,2000,1117.1,140,1112.22,2800,1117.40,4000,1112.00,1600,1117.5,100,1113.795,1111.565,1,2020-04-01 01:30:12.124381,2,1,30,12,1716.57,1.307440,1116.99,1117.5,1117.227894,1117.40,1117.10,1117.10,1117.40,1113.00,1116.00,1114.213404,1113.00,1113.00,1113.01,1116.00,1116.495,1115.880,1115.258333,NaN,NaN,0.869741,1.240013,NaN,NaN,3,-4416.0,0.126928,35.0,5.0,-30.0,1115.00,1116.333333,NaN,1.154701,NaN,1115.00,1116.860000,1117.020000,1.610931,1.219213,1117.81,1115.00,1117.00,1117.000,1117.2325,1117.011458,1117.00,1117.00,1113.50,1115.50,1117.00,1117.00,1116.083333
4,141,1585704615123840,1111.51,100,1111.62,2200,1111.50,200,1116.00,100,1111.30,100,1116.5,100,1111.21,3000,1117.00,300,1111.18,200,1117.4,4000,1111.565,1114.200,1,2020-04-01 01:30:15.123840,2,1,30,15,-1978.52,0.645332,1114.44,1117.5,1117.134810,1117.40,1117.40,1117.40,1117.40,1112.00,1113.15,1112.488630,1112.22,1112.22,1112.22,1113.00,1113.795,1115.145,1115.185000,1114.8925,NaN,1.909188,1.351777,1.24917,NaN,4,-1900.0,0.136364,17.0,3.0,-14.0,1113.50,1114.966667,NaN,1.270171,NaN,1116.95,1115.983333,1115.496667,0.837158,0.751443,1117.99,1113.01,1115.50,1117.810,1117.9400,1116.600476,1117.99,1115.70,1113.50,1114.60,1115.70,1115.70,1114.966667


In [34]:
feature_data['ts_id'].nunique()  

42007

In [36]:
# 處理nan值
#pd.set_option('display.max_rows', 500) #大部分五檔報價沒有對應到交易量
#pd.set_option('display.max_rows', 10)
print(feature_data.isna().sum())

index               0
TimeStamp           0
Bid1                0
Bid1Size            0
Ask1                0
                ...  
sell_min        40042
pre_sell_q1     40067
pre_sell_q2     40067
pre_sell_q3     40067
pre_sell_avg    40067
Length: 84, dtype: int64


In [38]:
#將na值填為-1
feature_data = feature_data.fillna(-1)
feature_data.head()

,index,TimeStamp,Bid1,Bid1Size,Ask1,Ask1Size,Bid2,Bid2Size,Ask2,Ask2Size,Bid3,Bid3Size,Ask3,Ask3Size,Bid4,Bid4Size,Ask4,Ask4Size,Bid5,Bid5Size,Ask5,Ask5Size,midpt,nextMidpt,model,TimeStamp2,Week,Hour,Minute,Second,book_size_diff,book_size_ratio,pre_ask_min,pre_ask_max,pre_ask_avg,pre_book_ask_mode,pre_book_ask_q1,pre_book_ask_q2,pre_book_ask_q3,pre_bid_min,pre_bid_max,pre_bid_avg,pre_book_bid_mode,pre_book_bid_q1,pre_book_bid_q2,pre_book_bid_q3,book_ma_6s,book_ma_12s,book_ma_24s,book_ma_48s,book_ma_96s,book_mstd_12s,book_mstd_24s,book_mstd_48s,book_mstd_96s,ts_id,trade_size_diff,trade_size_ratio,trade_buy_cnt,trade_sell_cnt,trade_cnt_diff,sell_ma1,sell_ma3,sell_ma6,sell_mstd3,sell_mstd6,buyl_ma1,buyl_ma3,buyl_ma6,buyl_mstd3,buyl_mstd6,buy_max,buy_min,pre_buy_q1,pre_buy_q2,pre_buy_q3,pre_buy_avg,pre_buy_mode,sell_max,sell_min,pre_sell_q1,pre_sell_q2,pre_sell_q3,pre_sell_avg
0,137,1585704603505567,1113.00,1900,1115.03,100,1112.22,2800,1116.00,500,1112.00,600,1117.0,100,1111.21,3000,1117.99,8358,1111.11,1600,1118.0,1300,1114.015,1115.265,1,2020-04-01 01:30:03.505567,2,1,30,3,4315.98,1.772916,-1.00,-1.0,-1.000000,1117.99,1117.99,1117.99,1117.99,-1.00,-1.00,-1.000000,1111.21,1111.21,1112.00,1112.22,-1.000,-1.000,-1.000000,-1.0000,-1.0,-1.000000,-1.000000,-1.00000,-1.0,0,2500.0,5.166667,6.0,8.0,2.0,1113.01,1114.336667,1114.668333,1.148927,0.812414,1115.99,1115.740000,1115.703333,0.245153,0.242872,1116.00,1115.50,1115.50,1115.615,1115.9250,1115.703333,1115.50,1115.50,1113.01,1115.00,1115.00,1115.00,1114.951935
1,138,1585704606961840,1115.03,100,1115.50,100,1114.44,200,1117.00,940,1113.01,1000,1117.1,3200,1113.00,1900,1117.40,4000,1112.22,2800,1117.5,100,1115.265,1116.495,1,2020-04-01 01:30:06.961840,2,1,30,6,415.50,1.074402,1115.03,1118.0,1117.857059,1117.99,1117.99,1117.99,1117.99,1111.11,1113.00,1111.870909,1111.21,1111.21,1112.00,1112.22,1114.015,-1.000,-1.000000,-1.0000,-1.0,-1.000000,-1.000000,-1.00000,-1.0,1,-3000.0,0.032258,19.0,1.0,-18.0,1113.11,-1.000000,-1.000000,-1.000000,-1.000000,1115.50,1115.500000,1115.725000,0.000000,0.551135,1116.85,1113.01,1113.01,1115.490,1115.5000,1114.798065,1113.01,1113.11,1113.11,1113.11,1113.11,1113.11,1113.110000
2,139,1585704609513298,1116.00,1600,1116.99,100,1115.03,300,1117.00,1240,1114.44,200,1117.1,3200,1113.01,600,1117.40,4000,1113.00,2000,1117.5,100,1116.495,1113.795,1,2020-04-01 01:30:09.513298,2,1,30,9,-885.99,0.841391,1115.50,1117.5,1117.218225,1117.40,1117.10,1117.10,1117.40,1112.22,1115.03,1112.719500,1112.22,1112.22,1113.00,1113.00,1115.265,1114.640,-1.000000,-1.0000,-1.0,0.883883,-1.000000,-1.00000,-1.0,2,-2200.0,0.083333,19.0,2.0,-17.0,1115.00,-1.000000,-1.000000,-1.000000,-1.000000,1117.00,1116.993333,1116.991667,0.005774,0.004082,1117.00,1115.40,1116.85,1116.850,1116.9300,1116.648182,1116.85,1115.00,1113.00,1113.50,1114.00,1114.50,1114.000000
3,140,1585704612124381,1113.15,300,1114.44,400,1113.01,600,1116.99,100,1113.00,2000,1117.1,140,1112.22,2800,1117.40,4000,1112.00,1600,1117.5,100,1113.795,1111.565,1,2020-04-01 01:30:12.124381,2,1,30,12,1716.57,1.307440,1116.99,1117.5,1117.227894,1117.40,1117.10,1117.10,1117.40,1113.00,1116.00,1114.213404,1113.00,1113.00,1113.01,1116.00,1116.495,1115.880,1115.258333,-1.0000,-1.0,0.869741,1.240013,-1.00000,-1.0,3,-4416.0,0.126928,35.0,5.0,-30.0,1115.00,1116.333333,-1.000000,1.154701,-1.000000,1115.00,1116.860000,1117.020000,1.610931,1.219213,1117.81,1115.00,1117.00,1117.000,1117.2325,1117.011458,1117.00,1117.00,1113.50,1115.50,1117.00,1117.00,1116.083333
4,141,1585704615123840,1111.51,100,1111.62,2200,1111.50,200,1116.00,100,1111.30,100,1116.5,100,1111.21,3000,1117.00,300,1111.18,200,1117.4,4000,1111.565,1114.200,1,2020-04-01 01:30:15.123840,2,1,30,15,-1978.52,0.645332,1114.44,1117.5,1117.134810,1117.40,1117.40,1117.40,1117.40,1112.00,1113.15,1112.488630,1112.22,1112.22,1112.22,1113.00,1113.795,1115.145,1115.185000,1114.8925,-1.0,1.909188,1.351777,1.24917,-1.0,4,-1900.0,0.136364,17.0,3.0,-14.0,1113.50,1114.966667,-1.000000,1.270171,-1.0000

In [40]:
#刪除不需要的欄位
drop_column = ['index','TimeStamp','TimeStamp2']
feature_data = feature_data.drop(columns = drop_column)
feature_data.head()

In [42]:
# 模型預測
# 1.資料分為train、test
x_train = feature_data.query('model==1').drop(columns='nextMidpt')
y_train = feature_data.query('model==1')['nextMidpt']

x_test = feature_data.query('model==0').drop(columns='nextMidpt') 
y_test = feature_data.query('model==0')['nextMidpt']

# from sklearn.model_selection import train_test_split
# x =  feature_data.query('model==1').drop(columns='nextMidpt')
# y =  feature_data.query('model==1')['nextMidpt']
# x_train,x_test,y_train,y_test = train_test_split(x, y, test_size=0.25, random_state=0)


# 2.調教模型超參數
from xgboost import XGBRegressor
from xgboost import XGBRFRegressor
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import mean_squared_error
# from sklearn.metrics import mean_absolute_error

from sklearn.preprocessing import Imputer
import math

def hyperparameter_tuning(x_train, y_train):
    
    param_grid = {'learning rate' = [0.01, 0.1],
                 'max_depth' = [3,5,7,10],
                 'min_child_weight' = [1,3,5],
                 'subsample': [0.5, 0.7],
                  'colsample_bytree': [0.5, 0.7],
                  'n_estimators' : [100, 200, 500],  #number of tree
                  'objective': ['reg:squarederror']}  #loss function
    
    estimator = XGBRegressor()
    
    gsearch = GridSearchCV(estimator = estimator,
                           param_grid = param_grid,
                           scoring = 'neg_root_mean_squared_error',  #RMSE  #https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
                           cv = 5,)
    
    return gsearch.best_params_

hyperparameter_tuning(x_train,y_train)

In [ ]:
# 3.最佳模型預測
xgb_model = XGBRegressor( objective = 'reg:squarederror',
                       colsample_bytree = 0.5,
                       learning_rate = 0.05,
                       max_depth = 6,
                       min_child_weight = 1,
                       n_estimators = 1000,
                       subsample = 0.7 )

%time 
xgb_model.fit(x_train, y_train)

y_pred = xgb_model.predict(x_test)


# 4.計算RMSE
MSE = mean_squared_error(y_test, y_pred)
RMSE = math.sqrt(MSE)
print("Root Mean Square Error:",RMSE)